In [3]:
import os
import cv2
import numpy as np

# Class map from pixel values to class names
class_map = {
    1: "chapati",
    2: "chiken",
    3: "curd",
    4: "curry",
    5: "dal",
    6: "idli",
    7: "paneer",
    8: "papad",
    9: "rice",
    10: "salad",
    11: "subji"
}

def extract_patches_from_mask(image_path, mask_path, output_dir):
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"❌ Could not load image: {image_path}")
        return
    if mask is None:
        print(f"❌ Could not load mask: {mask_path}")
        return

    object_ids = np.unique(mask)
    object_ids = object_ids[object_ids != 0]  # exclude background (0)

    for obj_id in object_ids:
        obj_mask = (mask == obj_id).astype(np.uint8)

        contours, _ = cv2.findContours(obj_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for i, contour in enumerate(contours):
            x, y, w, h = cv2.boundingRect(contour)
            patch = image[y:y+h, x:x+w]

            class_name = class_map.get(obj_id, f"class_{obj_id}")
            class_folder = os.path.join(output_dir, class_name)
            os.makedirs(class_folder, exist_ok=True)

            base_name = os.path.splitext(os.path.basename(image_path))[0]
            patch_filename = f"{base_name}_cls{obj_id}_{i}.png"
            patch_path = os.path.join(class_folder, patch_filename)
            cv2.imwrite(patch_path, patch)

# Set your directories here
image_dir = r"C:\Users\ADAPA SUNIL KUMAR\Desktop\AIG FOOD PROJECT\train\train\images"
mask_dir = r"C:\Users\ADAPA SUNIL KUMAR\Desktop\AIG FOOD PROJECT\train\train\masks"
output_dir = r"C:\Users\ADAPA SUNIL KUMAR\Desktop\AIG FOOD PROJECT\train\train\output"

# Process all image-mask pairs
for image_filename in os.listdir(image_dir):
    if image_filename.endswith(".jpg") or image_filename.endswith(".png"):
        image_path = os.path.join(image_dir, image_filename)

        # Build mask filename (assumes _mask.png format)
        base_name = os.path.splitext(image_filename)[0]
        mask_filename = base_name + "_mask.png"
        mask_path = os.path.join(mask_dir, mask_filename)

        extract_patches_from_mask(image_path, mask_path, output_dir)
